# SQL Server 2019 Data Virtualization - Using Polybase to query Azure CosmosDB
This notebook contains an example of how to use external tables to query data in Azure CosmosDB (using MongoDB API) without moving data. You may need to change identity, secret, connection, database, schema, and remote table names to work with your Azure CosmosDB.

This notebook also assumes you are using SQL Server 2019 Release Candidate or later and that the Polybase feature has been installed and enabled.

This notebook uses the sample WideWorldImporters sample database but can be used with any user database.

## Step 0: Create the CosmosDB database, document, collection, and add data

Create a new database, collection, and document with CosmosDB in Azure. You can the Azure portal to create a new Azure CosmosDB database (choose for Mongo API). Use the Data Explorer tool from the portal to create a database called **WideWorldImporters** with a collection called **Orders**. Then create a new document with field names and values like the following (Note: the _id field was created by Data Explorer and the id field was a default value already provided by the tool)


```json
{
	"_id" : ObjectId("5c54aa72dd13c70f445745bf"),
	"id" : "1",
	"OrderID" : 1,
	"SalesPersonPersonID" : 2,
	"CustomerName" : "Vandelay Industries",
	"CustomerContact" : "Art Vandelay",
	"OrderDate" : "2018-05-14",
	"CustomerPO" : "20180514",
	"ExpectedDeliveryDate" : "2018-05-21"
}
```

## Step 1: Create a master key
Create a master key to encrypt the database credential

In [1]:
USE [WideWorldImporters]
GO
CREATE MASTER KEY ENCRYPTION BY PASSWORD = '<password>'
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.018

## Step 2: Create a database credential
Create the database scoped credentials with the Azure CosmosDB user and password. You can get the IDENTITY (user) and secret (password) from the Connection String option in the Azure portal

In [2]:
CREATE DATABASE SCOPED CREDENTIAL CosmosDBCredentials   
WITH IDENTITY = '<user>', Secret = '<password>'
GO

Commands completed successfully.

Total execution time: 00:00:00.004

## Step 3: Create an EXTERNAL DATA SOURCE
The EXTERNAL DATA SOURCE indicates what type of data source, the connection "string", where PUSHDOWN predicates should be used (if possible), and the name of the database credential.

The LOCATION syntax is <datasourcetype>:<connection string>.

datasourcetype can be sqlserver, oracle, teradata, mongodb, or odbc (Windows only)
The connection string depends on the datasourcetype

For this example, create a data source for the Azure CosmoDB sderver using the host URI and port. The LOCATION is built from <HOST>:<PORT> from the Connection String in the Azure Portal

In [3]:
CREATE EXTERNAL DATA SOURCE CosmosDB
WITH ( 
LOCATION = 'mongodb://<uri>:<port>',
PUSHDOWN = ON,
CREDENTIAL = CosmosDBCredentials
)
GO

Commands completed successfully.

Total execution time: 00:00:00.027

## Step 4: Create a schema for the EXTERNAL TABLE
Schemas provide convenient methods to secure and organize objects

In [4]:
CREATE SCHEMA cosmosdb
GO

Commands completed successfully.

Total execution time: 00:00:00.004

## Step 5: Create an EXTERNAL TABLE
An external table provides metadata so SQL Server knows how to map columns to the remote table. The name of the table for the external table can be your choice. But the columns must be specified with the same name as they are defined in the remote table. Furthermore, local data types must be compatible with the remote table.

Create the external table to match the Azure CosmosDB document. The WITH clause specifies a LOCATION. This LOCATION is different than the EXTERNAL DATA SOURCE. The LOCATION is the CosmosDB database and collection.

In [7]:
CREATE EXTERNAL TABLE cosmosdb.Orders
(
	[_id] NVARCHAR(100) COLLATE Latin1_General_100_CI_AS NOT NULL,
	[id] NVARCHAR(100) COLLATE Latin1_General_100_CI_AS NOT NULL,
	[OrderID] int NOT NULL,
	[SalesPersonPersonID] int NOT NULL,
	[CustomerName] NVARCHAR(100) COLLATE Latin1_General_100_CI_AS NOT NULL,
	[CustomerContact] NVARCHAR(100) COLLATE Latin1_General_100_CI_AS NOT NULL,
	[OrderDate] NVARCHAR(100) COLLATE Latin1_General_100_CI_AS NOT NULL,
	[CustomerPO] NVARCHAR(100) COLLATE Latin1_General_100_CI_AS NULL,
	[ExpectedDeliveryDate] NVARCHAR(100) COLLATE Latin1_General_100_CI_AS NOT NULL
)
 WITH (
 LOCATION='WideWorldImporters.Orders',
 DATA_SOURCE=CosmosDB
)
GO

Commands completed successfully.

Total execution time: 00:00:00.335

## Step 6: Create statistics
SQL Server allows you to store local statistics about specific columns from the remote table. This can help the query processing to make more efficient plan decisions.

In [8]:
CREATE STATISTICS CosmosDBOrderSalesPersonStats ON cosmosdb.Orders ([SalesPersonPersonID]) WITH FULLSCAN
GO

Commands completed successfully.

Total execution time: 00:00:01.793

## Step 7: Try to scan the remote table
Run a simple query on the EXTERNAL TABLE to scan all rows.

In [9]:
SELECT * FROM cosmosdb.Orders
GO

(1 row affected)

Total execution time: 00:00:00.535

_id,id,OrderID,SalesPersonPersonID,CustomerName,CustomerContact,OrderDate,CustomerPO,ExpectedDeliveryDate
5c54aa72dd13c70f445745bf,1,1,2,Vandelay Industries,Art Vandelay,2018-05-14,20180514,2018-05-21


## Step 8: Query the remote table with a WHERE clause
Even though the table may be small SQL Server will "push" the WHERE clause filter to the remote table

In [10]:
SELECT * FROM cosmosdb.Orders WHERE SalesPersonPersonID = 2
GO

(1 row affected)

Total execution time: 00:00:01.343

_id,id,OrderID,SalesPersonPersonID,CustomerName,CustomerContact,OrderDate,CustomerPO,ExpectedDeliveryDate
5c54aa72dd13c70f445745bf,1,1,2,Vandelay Industries,Art Vandelay,2018-05-14,20180514,2018-05-21


## Step 9: Join with local SQL Server tables
Find out the name of the salesperson and which customer they worked with to test out the new mobile app experience.

In [11]:
SELECT FullName, o.CustomerName, o.CustomerContact, o.OrderDate
FROM cosmosdb.Orders o
JOIN [Application].[People] p
ON o.SalesPersonPersonID = p.PersonID
GO

(1 row affected)

Total execution time: 00:00:00.452

FullName,CustomerName,CustomerContact,OrderDate
Kayla Woodcock,Vandelay Industries,Art Vandelay,2018-05-14
